In [8]:
import logging
logging.getLogger("httpx").setLevel(logging.WARNING)

import pandas as pd
from tqdm.auto import tqdm

job_id='dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc'

df = pd.read_csv(f'../../extractions/{job_id}/relevant_articles_to_classify_2025.csv',encoding='utf-8')
df

,source_name,title,company_codes,modification_date,region_codes,segments,num_segments,an,tile_id,company_names
0,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz",Enhances Cleanova’s presence in the UK and bro...,4,GUONGA0020250109el170007b,1910620,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
1,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz","\n\nCleanova, a clean tech, market-leading glo...",4,GUONGA0020250109el170007b,1910621,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
2,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz","\n\nCleanova’s CEO, Javaid Riaz, commented, “T...",4,GUONGA0020250109el170007b,1910622,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
3,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz",\n\nJavaid Riaz explained what the transition ...,4,GUONGA0020250109el170007b,1910623,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
4,thesun.co.uk,HIGH STREET GIANT WHSmith ‘in secret talks to ...,"bhchcm,brraic,bsbro,lsexch,smwh",2025-01-26 07:33:12.758,"eurz,uk,weurz",The retail group have been in negotiations wit...,7,THESUK0020250125el1p001jl,1910746,"British Chambers of Commerce,British Retail Co..."
...,...,...,...,...,...,...,...,...,...,...
135280,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nJP Morgan was completely unaware that Coat...,7,TIMEUK0020250503el53000jh,2940030,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135281,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nBet365 has historically had a sizeable rem...,7,TIMEUK0020250503el53000jh,2940031,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135282,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nBut a sale risks bringing Coates into conf...,7,TIMEUK0020250503el53000jh,2940032,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135283,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz","\n\nAccordingly a sale may be her only option,...",7,TIMEUK0020250503el53000jh,2940033,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."


In [9]:
business_request = """
### Deals search criteria
* Completed date (last 10 years)
* Geography (Europe)

* Deal technique (Divestment)
* Sector (Financial services)
* Size (TBD)

Note that we are looking for future carve-outs, so please do not include any deals that have already been completed.
"""
def load_examples(examples_file="../../data/few_shot_examples.csv", input_cols = ['an', 'news_source',"article_title","article_body"], output_cols=["is_co", "comment"]):
    df = pd.read_csv(examples_file,encoding='utf-8', sep=';')
    examples = df.to_dict(orient='records')
    examples = [{"inputs": {col: ex[col].strip('\n\n') for col in input_cols},
                "outputs": {col: ex[col] for col in output_cols}} for ex in examples]
    return examples

# examples = load_examples()
# examples

In [10]:
from typing import List, Dict, Any

def df_to_prompt_inputs(df: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Convert a DataFrame with columns ['source_name', 'title', 'body', 'company_names','company_codes','modification_date]
    into a list of dicts compatible with your prompt template keys:
      - news_source
      - article_title
      - article_body
      - mentioned_companies
      - mentioned_company_codes
      - modification_date
    """
    df_renamed = df.rename(columns={
        'source_name': 'news_source',
        'title': 'article_title',
        'segments': 'article_body',
        'company_names': 'companies',
        'company_codes': 'company_codes',
        'modification_date': 'modification_date'
    })
    df_renamed['business_request'] = business_request
    return df_renamed[
        ['an', 'news_source', 'article_title', 'article_body', 'companies', 'company_codes','modification_date', 'business_request']
    ].reset_index().to_dict(orient='records')
    
def chunk_list(items: List[Any], batch_size: int) -> List[List[Any]]:
    """
    Split a list into sublists (batches) of max length `batch_size`.
    """
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

chunks = chunk_list(df_to_prompt_inputs(df), 100)
chunks[0][:5]

[{'index': 0,
  'an': 'GUONGA0020250109el170007b',
  'news_source': 'Gulf Oil & Gas',
  'article_title': 'Cleanova expands its product offering with the acquisition of Allied Filter',
  'article_body': 'Enhances Cleanova’s presence in the UK and broader European market\n\nHighlights its commitment to growth, innovation and quality in industrial filtration',
  'companies': 'Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimpton,Perkins Coie LLP',
  'company_codes': 'alvmar,cmbndi,dbvpl,percoi',
  'modification_date': '2025-01-10 07:13:57.825',
  'business_request': '\n### Deals search criteria\n* Completed date (last 10 years)\n* Geography (Europe)\n\n* Deal technique (Divestment)\n* Sector (Financial services)\n* Size (TBD)\n\nNote that we are looking for future carve-outs, so please do not include any deals that have already been completed.\n'},
 {'index': 1,
  'an': 'GUONGA0020250109el170007b',
  'news_source': 'Gulf Oil & Gas',
  'article_title': 'Cleanova expands its product offering wi

In [11]:
from typing import Literal
from pydantic import BaseModel,Field

class CarveOutAssessment(BaseModel):
    is_co: bool = Field(description="Is the article about a future corporate carve-out?")
    target_company_code: str = Field(description="Code of the company that may be open to divest/carve-out its subsidiary")
    subsidiary_company_code: str = Field(description="Code of the subsidiary that may be open to divest/carve-out")
    is_relevant: bool = Field(description="Is the article relevant to the conditions of business request?")
    co_stage: Literal["Early", "Late"] = Field(description="Stage of the carve-out opportunity")
    short_reasoning: str = Field(description="Justification for the answers provided; not more than 2 sentences")

template = """
You are an expert financial analyst specializing in identifying potential corporate carve-out opportunities in the financial services sector.
Your task is to analyze news articles to determine whether they indicate a company is considering divesting part of its business in the future.
Focus on European financial services companies across ALL segments including banking, wealth management, insurance, financial data,
payment processing, fund administration, brokers, and other financial services.
    
CRITICAL EVALUATION CRITERIA:
1. MUST IDENTIFY AS CARVE-OUT PROSPECT:
   - Company signals intention to divest specific business units or segments
   - Company announces strategic refocusing that implies non-core units could be sold
   - New strategic plans mentioning focus on specific segments (implying others might be divested)
   - Language about simplifying corporate structure or streamlining operations
   - References to non-core assets, underperforming units, or subscale operations
   - Management discussing 'evaluating options' for specific business segments
    
2. MUST NOT BE INCORRECTLY CLASSIFIED AS CARVE-OUT:
   - Simple stake sales without business unit separation 
   - Private equity firms exiting investments (not corporate carve-outs)
   - IPO plans without specific business unit divestiture
   - Already completed carve-outs (though mention these in reasoning as potentially interesting)
   - General M&A activity without specific divestiture signals
   - Opinion pieces or columns without factual business announcements
    
3. TIMING RELEVANCE:
   - FOCUS ON FUTURE OPPORTUNITIES: Companies currently considering or likely to consider divestitures
   - DE-PRIORITIZE: Already completed transactions (though note these in reasoning)
    
4. KEY SIGNALS OF POTENTIAL CARVE-OUTS:
   - New CEO appointments or management changes
   - Announcements of strategic reviews or new strategic plans
   - Explicit focus on 'core businesses' or 'key segments'
   - Financial pressure (debt issues, dividend concerns, performance challenges)
   - Regulatory challenges that might prompt divestiture
   - Simplification of corporate structure
   - Discontinuation of operations in certain areas
    
When selecting target_company_code:
- If multiple companies might be considering divestitures, select the primary/most likely one
- For corporate groups where a parent may divest subsidiaries, consider both parent and subsidiary codes
- If a company is refocusing on specific segments, this implies other segments could be divested
- Do not pick the wrong company - target company is the one that may divest a subsidiary, a subsidiary is the one that may be divested
    
Business request: {business_request}

Analyze this article and provide a structured assessment of potential carve-out opportunities:
news source: {news_source}
article body: {article_body}
companies: {companies}
company codes: {company_codes}
"""

In [12]:
from llm_utils.factory import construct_llm, LLMChainFactory
llm = construct_llm(model_name='gpt-4.1-mini', temperature=0.1)
factory = LLMChainFactory(model_name='gpt-4.1-mini', temperature=0.1)
so_chain = factory._runnable_with_structured(template=template, response_schemas=CarveOutAssessment, llm=llm, business_request=business_request)

### Token Usage & Cost Analysis

In [ ]:
from langchain.callbacks import get_openai_callback

def simple_batch_with_usage(batch, chain):
    # Wrap the call in the OpenAI callback context
    with get_openai_callback() as cb:
        results = chain.batch(batch)
    # After the call, cb has all the stats
    # print(f"Prompt tokens:  {cb.prompt_tokens}")
    # print(f"Completion tokens: {cb.completion_tokens}")
    # print(f"Total tokens:   {cb.total_tokens}")
    return results, cb

def batch_with_usage(batch, chain):
    # Wrap the call in the OpenAI callback context
    with get_openai_callback() as cb:
        results = chain.batch(inputs=batch)
    # After the call, cb has all the stats
    # print(f"Prompt tokens:  {cb.prompt_tokens}")
    # print(f"Completion tokens: {cb.completion_tokens}")
    # print(f"Total tokens:   {cb.total_tokens}")
    return results, cb

def batches_with_tokens(batches, chain):
    """
    Process a list of batches with the provided chain and return the results along with token usage.
    """
    total_prompt_tokens, total_completion_tokens, total_tokens = 0, 0, 0

    all_results = []
    for batch in tqdm(batches):
        results, cb = batch_with_usage(batch, chain)
        all_results.extend(results)
        total_prompt_tokens += cb.prompt_tokens
        total_completion_tokens += cb.completion_tokens
        total_tokens += cb.total_tokens
    print(f"Total prompt tokens across {len(batches)} batches:  {total_prompt_tokens}")
    print(f"Total completion tokens across {len(batches)} batches: {total_completion_tokens}")
    print(f"Total tokens across {len(batches)} batches:   {total_tokens}")
    return all_results

co_batch_res = batches_with_tokens(chunks[:10], so_chain)

def price_per_dataset(input_toks_price, output_toks_price, input_toks_per_batch, output_toks_per_batch, chunks):
    input_toks_norm = input_toks_per_batch / 1000000
    output_toks_norm = output_toks_per_batch / 1000000
    
    price_per_batch = input_toks_price * input_toks_norm + output_toks_price * output_toks_norm
    total_price = price_per_batch * len(chunks)
    return round(total_price,2)


  0%|          | 0/10 [00:00<?, ?it/s]

INFO:openai._base_client:Retrying request to /chat/completions in 1.000000 seconds


In [ ]:
price_input_toks_per_mln = 0.4
price_output_toks_per_mln = 1.6

input_toks_per_10_batches = 690582
input_toks_per_10_batches_mln = input_toks_per_10_batches/1000000

output_toks_per_10_batches = 45113
output_toks_per_10_batches_mln = output_toks_per_10_batches/1000000


price_per_10_batches = price_input_toks_per_mln * input_toks_per_10_batches_mln + price_output_toks_per_mln * output_toks_per_10_batches_mln
price_per_batch = price_per_10_batches / 10

total_price_mini = price_per_batch * len(chunks[:]) #total price for 135k entries
round(total_price_mini, 2), price_per_dataset(price_input_toks_per_mln,price_output_toks_per_mln, input_toks_per_10_batches/10, output_toks_per_10_batches/10, chunks)

(78.57, 78.57)

In [ ]:
total_price_nano = price_per_dataset(0.1, 0.4, input_toks_per_10_batches/10, output_toks_per_10_batches/10, chunks)
total_price_nano

19.64

In [ ]:
import time
from typing import List, Dict, Any
from pathlib import Path
from datetime import datetime
import pandas as pd
from tqdm.auto import tqdm

async def process_and_save(
    chunks: List[List[Dict[str, Any]]],
    chain,
    partial_every: int = 10,
    partial_dir: str = "partials",
    final_path: str = "classification_results.csv",
    max_retries: int = 5,
    initial_delay: float = 5.0,
) -> pd.DataFrame:
    """
    Iterate over each chunk of inputs, run the chain with retry logic, and save results:
      - Shows a progress bar over chunks.
      - Retries on failure with exponential backoff.
      - Saves partial CSV after every `partial_every` chunks into `partial_dir`.
      - Saves the final combined results as `final_path`.
      - Uses company codes from input chunks instead of LLM responses.
    
    Args:
        chunks: List of batches of prompt-input dicts.
        chain: RunnableSequence with .batch().
        partial_every: Save a partial CSV every N chunks.
        partial_dir: Directory to save partial CSVs.
        final_path: Filepath for the final CSV.
        max_retries: How many times to retry a failed chunk.
        initial_delay: Base delay (in seconds) for exponential backoff.
    
    Returns:
        DataFrame of all successful results.
    """
    partial_dir = f"{partial_dir}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
    Path(partial_dir).mkdir(parents=True, exist_ok=True)
    all_records = []
    failed_chunks = []

    for idx, batch in enumerate(tqdm(chunks, desc="Processing chunks")):
        # Retry logic per chunk
        for attempt in range(1, max_retries + 1):
            try:
                assessments = await chain.abatch(batch)
                break
            except Exception as e:
                print(f"[Chunk {idx}][Attempt {attempt}] Error: {e}")
                if attempt < max_retries:
                    delay = initial_delay * (2 ** (attempt - 1))
                    print(f"  Retrying in {delay:.1f}s...")
                    time.sleep(delay)
                else:
                    print(f"  Chunk {idx} failed after {max_retries} attempts, skipping.")
                    failed_chunks.append(idx)
        else:
            # All retries failed: skip to next chunk
            continue

        # On success, create records and enrich with original data from chunks
        records = []
        for i, assessment in enumerate(assessments):
            record = assessment.model_dump()
            
            # Add original company codes and any other metadata from the input chunk
            record['original_company_codes'] = batch[i].get('company_codes', '')
            record['original_companies'] = batch[i].get('companies', '')
            record['index'] = batch[i].get('index', None)

            records.append(record)
            
        all_records.extend(records)

        # Save partials
        if (idx + 1) % partial_every == 0:
            df_partial = pd.DataFrame(all_records)
            df_partial.to_csv(Path(partial_dir) / f"partial_{idx+1}.csv", index=False)
            print(f"Saved partial results to {partial_dir}/partial_{idx+1}.csv")
        time.sleep(5)
    
    # Save final results
    df_final = pd.DataFrame(all_records)
    final_file = Path(f"{final_path}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")
    final_file.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(final_file, index=False)
    print(f"Saved final results to {final_file}")

    if failed_chunks:
        print(f"Warning: the following chunks failed and were skipped: {failed_chunks}")
    
    return df_final

In [ ]:
df_results = await process_and_save(
    chunks[:],
    so_chain,
    partial_every=10,
    partial_dir=f"../../extractions/{job_id}/partial_outputs",
    final_path=f"../../extractions/{job_id}/co_classification_results.csv",
    max_retries=4,
    initial_delay=5.0)

In [ ]:
merged_df = pd.merge(
    df, 
    df_results,
    on=['an'],  # Adjust these keys as needed
    how='inner',
    suffixes=('_original', '_new')
)

In [ ]:
merged_df["is_co"] = merged_df["is_co"].astype(int)
merged_df["is_relevant_new"] = merged_df["is_relevant_new"].astype(int)

### Calculating metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# For carve-out detection metrics
y_true_co = merged_df['is_about_carve_out']  # Ground truth
y_pred_co = merged_df['is_co']  # Predictions

co_metrics = {
    'accuracy': accuracy_score(y_true_co, y_pred_co),
    'precision': precision_score(y_true_co, y_pred_co),
    'recall': recall_score(y_true_co, y_pred_co),
    'f1': f1_score(y_true_co, y_pred_co),
    'confusion_matrix': confusion_matrix(y_true_co, y_pred_co)
}

# For relevance detection metrics
y_true_rel = merged_df['is_relevant_original']  # Ground truth 
y_pred_rel = merged_df['is_relevant_new']  # Predictions

rel_metrics = {
    'accuracy': accuracy_score(y_true_rel, y_pred_rel),
    'precision': precision_score(y_true_rel, y_pred_rel),
    'recall': recall_score(y_true_rel, y_pred_rel),
    'f1': f1_score(y_true_rel, y_pred_rel),
    'confusion_matrix': confusion_matrix(y_true_rel, y_pred_rel)
}

# Print results
print("Carve-out Detection Metrics:")
for metric, value in co_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric}: {value:.4f}")
print(f"Confusion Matrix:\n{co_metrics['confusion_matrix']}")

print("\nRelevance Detection Metrics:")
for metric, value in rel_metrics.items():
    if metric != 'confusion_matrix':
        print(f"{metric}: {value:.4f}")
print(f"Confusion Matrix:\n{rel_metrics['confusion_matrix']}")

Carve-out Detection Metrics:
accuracy: 0.9143
precision: 0.8433
recall: 0.8370
f1: 0.8401
Confusion Matrix:
[[346  21]
 [ 22 113]]

Relevance Detection Metrics:
accuracy: 0.7948
precision: 0.2966
recall: 0.9773
f1: 0.4550
Confusion Matrix:
[[356 102]
 [  1  43]]


### Extracting more phrases

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


class CarveOutPhrases(BaseModel):
    """Structured output for identifying intent and action phrases in financial news articles."""
    
    search_phrases: List[str] = Field(
        description="List of concise 3-6 word phrases that combine intent and action related to divestitures",
        default_factory=list
    )


def build_runnable_with_pydantic(template, pydantic_model, llm, **kwargs):
    output_parser = PydanticOutputParser(pydantic_object=pydantic_model)
    prompt = PromptTemplate.from_template(template, partial_variables=kwargs)
    output_fixing_parser = OutputFixingParser.from_llm(
        llm=llm, parser=output_parser
    )
    return prompt | llm | output_fixing_parser


carve_out_template = """
You are a financial analyst with expertise in corporate divestitures, carve-outs, and spin-offs.

Your task is to analyze the following financial news article and extract concise 3-6 word phrases that capture both the intent and action related to potential carve-outs or divestitures.

# ARTICLE TITLE:
{title}

# ARTICLE TEXT:
{body}

# EXAMPLES OF GOOD PHRASES:
Intent-focused phrases:
- "plan to divest"
- "intend to sell"
- "looking to dispose"
- "considering strategic options"
- "mulling asset sale"
- "weighing divestiture options"
- "strategic review of assets"
- "evaluating options for division"
- "exploring alternatives for subsidiary"
- "assessing future of unit"
- "not core to strategy"
- "exploring strategic options to"

Action-focused phrases:
- "divest non-core assets"
- "sell business unit"
- "spin-off division"
- "dispose of subsidiary"
- "carve-out regional operations"
- "exit non-strategic markets"
- "separate underperforming businesses"
- "monetize legacy assets"
- "offload international operations"
- "part ways with division"

# INSTRUCTIONS:
- Create concise 3-6 word phrases that combine both intent and action elements
- Focus on the exact language used in this specific article
- Extract phrases that would be useful for finding similar articles
- Determine if this article discusses a potential/actual carve-out or divestiture
- Rate confidence that this describes a real carve-out opportunity (1-5)

# OUTPUT FORMAT:
Extract phrases that are concise (3-6 words) but still capture the key aspects of the potential transaction.
Each phrase should be useful as a search term for finding similar articles about carve-outs.
"""

def create_carve_out_extraction_chain(llm):
    return build_runnable_with_pydantic(
        template=carve_out_template,
        pydantic_model=CarveOutPhrases,
        llm=llm,
        format_instructions=PydanticOutputParser(pydantic_object=CarveOutPhrases).get_format_instructions()
    )

In [ ]:
from tqdm.auto import tqdm
# Example usage
def process_articles(df, llm):
    """
    Process each article in the dataframe to extract intent and action phrases
    
    Args:
        df: DataFrame containing financial articles
        llm: The language model to use for extraction
    
    Returns:
        Sets of unique intent and action phrases found across all articles
    """
    extraction_chain = create_carve_out_extraction_chain(llm)
    
    all_search_phrases = set()
    
    df = df[df['is_about_carve_out'] == 1]
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting phrases"):
        result = extraction_chain.invoke({
            "title": row['title'],
            "body": row['body']
        })
                
        all_search_phrases.update(result.search_phrases)
            
    return all_search_phrases

In [ ]:
a = process_articles(df, llm)

Extracting phrases: 100%|██████████| 135/135 [05:48<00:00,  2.58s/it]


In [ ]:
a

{'Altice must sell Portuguese operator',
 'BASF sells Brazilian coatings business',
 'BP mulls Castrol sale',
 'BP to offload Castrol',
 'Barclays exploring payments unit sale',
 'Barclays to sell payments arm',
 'Barclays to sell payments stake',
 'BlackRock trims AIB ownership',
 'Brookfield may acquire payments unit',
 'De La Rue takeover agreement',
 'HSBC considering exiting international markets',
 'HSBC reviewing retail banking operations',
 'IPO of 25-30% stake',
 'KKR may exit Northumbrian',
 'KKR must divest Northumbrian stake',
 'KKR reviewing water asset overlap',
 'Kantar to be broken up',
 'Macquarie faces Exolum divestment',
 'NAB divests MLC stake',
 'NAB strategic exit from MLC',
 'NAB to sell MLC stake',
 'NatWest full government exit plan',
 'Nexi mulls unit sale',
 'Nippon Life expansion via acquisition',
 'Nippon Life to acquire MLC',
 'Santander plans business spinoff',
 'Schroders looking to sell business',
 'Schroders weighs exiting sub-scale markets',
 'SocGen 